In [30]:
pip install --upgrade requests

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests in c:\users\dr e.a maji\anaconda33\lib\site-packages (2.31.0)



In [2]:
from bs4 import BeautifulSoup
import requests
import lxml

SCRAPING TITANIC TRANSCRIPT

In [5]:
#fetch the pages
website = "https://subslikescript.com/movie/Titanic-120338"
result = requests.get(website)

#Page content 
content = result.text

#Create Soup
soup = BeautifulSoup(content, "lxml")
#print(soup.prettify())

box = soup.find('article', class_="main-article")
title = box.find('h1').get_text()
transcript = box.find('div',class_='full-script').get_text(strip=True, separator=' ')

#with open(f'{title}.txt', 'w') as file:
  #  file.write(transcript)


print(title)
print(transcript)

Titanic (1997) - full transcript
foodval.com - stop by if you're interested in the nutritional composition of food --- 13 meters. You should see it. Okay, take her up and over the bow rail. Mir 2, we're going over the bow. Stay with us. Okay, quiet. We're rolling. Seeing her coming out of the darkness like a ghost ship... ...still gets me every time. To see the sad ruin of the great ship sitting here... ...where she landed at 2:30 in the morning Of April 15, 1912 ...after her long fall... ...from the world above. You are so full of shit, boss. Dive 6. Here we are again on the deck of Titanic. 2% miles down 3,821 meters- The pressure outside is 3% tons per square inch These windows are 9 inches thick. If they go, it's sayonara in two microseconds. All right, enough of that bullshit. Just put her down on the roof of the officers' quarters like yesterday. Mir 2, we're landing right over the Grand Staircase. You guys set to launch? Yeah, Brock. Launching Dunkin now. Go, Charlie. - All righ

SCAPING MULTIPLE LINKS WITHIN THE SAME PAGE

In [4]:
#fetch the pages
root = 'https://subslikescript.com'
website = f'{root}/movies'
result = requests.get(website)

#Page content 
content = result.text

#Create Soup
soup = BeautifulSoup(content, "lxml")
#print(soup.prettify())
box = soup.find('article', class_="main-article")

links = []
for link in box.find_all('a', href=True):
    links.append(link['href'])

print(links)

for link in links:
    website = f'{root}/{link}'
    result = requests.get(website)
    content = result.text
    soup = BeautifulSoup(content, "lxml")
    
    box = soup.find('article', class_="main-article")

    title = box.find('h1').get_text()
    transcript = box.find('div',class_='full-script').get_text(strip=True, separator=' ')

#with open(f'{title}.txt', 'w') as file:
 #   file.write(transcript)

['movie/Roads_of_Fear-7875666', 'movie/In_deulim-22505246', 'movie/The_Hut-220695', 'movie/_lattaque-235108', 'movie/Taylor_Swift_Speak_Now_World_Tour_Live-2103256', 'movie/Ltrange_dsir_de_Monsieur_Bard-46572', 'movie/Bad_Genius-6788942', 'movie/All_Roads_Lead_to_Rome-41977', 'movie/Indiscreet-51773', 'movie/The_Dream_of_Lu-2214997', 'movie/Amanda_Knox-5952332', 'movie/City_After_Dark-81115', 'movie/The_Allure_of_Tears-2170573', 'movie/A_Christmas_Carol_Goes_Wrong-7766092', 'movie/Vishwatma-130350', 'movie/Ghatak-4771744', 'movie/Night_Explorers_The_Asylum-14088788', 'movie/August_at_Twenty-Two-10988502', 'movie/The_Joy_of_Living-53707', 'movie/The_Incredible_Monk_-_Dragon_Return-28107464', 'movie/Cabin_Fever-303816', 'movie/See_You_on_Venus-14960612', 'movie/Ouija_Shark_2-21409958', 'movie/Moon_Garden-18561446', 'movie/Sun_Moon-14999006', 'movie/The_Constant_Nymph-24998', 'movie/xXx_State_of_the_Union-329774', 'movie/Los_Habitantes-16427598', 'movie/Brothers-3802576', 'movie/The_Black

In [8]:
import requests
from bs4 import BeautifulSoup

# Fetch the main page
root = 'https://subslikescript.com'
website = f'{root}/movies'
result = requests.get(website)

# Parse the main page
content = result.text
soup = BeautifulSoup(content, "lxml")
box = soup.find('article', class_="main-article")

# Extract links from the main page
links = [link['href'] for link in box.find_all('a', href=True)]

# Iterate over each link
for link in links:
    website = f'{root}/{link}'
    result = requests.get(website)
    content = result.text
    soup = BeautifulSoup(content, "lxml")
    
    # Parse the content of the page
    box = soup.find('article', class_="main-article")
    title = box.find('h1').get_text()
    transcript = box.find('div', class_='full-script').get_text(strip=True, separator=' ')

    #Save the transcript to a file 
    with open(f'{title}.txt', 'w', encoding='utf-8') as file:
        file.write(transcript) 


In [10]:
title

'The Black Pharaoh, the Savage and the Princess (2022) - full transcript'

PAGINATION: This is scraping different number of pages in a website

In [24]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup

# Function to get a session with retries
def get_session_with_retries(retries, backoff_factor, status_forcelist):
    session = requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('https://', adapter)
    session.mount('http://', adapter)
    return session

# Create a session
session = get_session_with_retries(retries=5, backoff_factor=0.3, status_forcelist=[500, 502, 503, 504])

# Fetch the main page
root = 'https://subslikescript.com'
website = f'{root}/movies_letter-A'
result = session.get(website)

# Parse the main page
content = result.text
soup = BeautifulSoup(content, "lxml")

# Pagination
pagination = soup.find('ul', class_='pagination')
pages = pagination.find_all('li', class_='page-item')
last_page = pages[-2].text

links = []
for page in range(1, int(last_page) + 1)[:2]:  # [1, 138+1] to cover all pages
    result = session.get(f'{website}?page={page}')
    content = result.text
    soup = BeautifulSoup(content, "lxml")

    box = soup.find('article', class_="main-article")

    # Extract links from the main page
    for a in box.find_all('a', href=True):
        links.append(root + a['href'])

# Iterate over each link
for link in links:
    try:
        print(link)
        result = session.get(link)
        content = result.text
        soup = BeautifulSoup(content, "lxml")

        # Parse the content of the page
        box = soup.find('article', class_="main-article")
        title = box.find('h1').get_text()
        transcript = box.find('div', class_='full-script').get_text(strip=True, separator=' ')

        with open(f'{title}.txt', 'w', encoding='utf-8') as file:
            file.write(transcript)

    except Exception as e:
        print('-------- Link not working ----------')
        print(link)
        print(e)


https://subslikescript.commovie/A_1000000000000000_Ransom-4926482
-------- Link not working ----------
https://subslikescript.commovie/A_1000000000000000_Ransom-4926482
HTTPSConnectionPool(host='subslikescript.commovie', port=443): Max retries exceeded with url: /A_1000000000000000_Ransom-4926482 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001C5BF15E1D0>: Failed to resolve 'subslikescript.commovie' ([Errno 11001] getaddrinfo failed)"))
https://subslikescript.commovie/A_2nd_Chance-2150139
-------- Link not working ----------
https://subslikescript.commovie/A_2nd_Chance-2150139
HTTPSConnectionPool(host='subslikescript.commovie', port=443): Max retries exceeded with url: /A_2nd_Chance-2150139 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001C5BF165310>: Failed to resolve 'subslikescript.commovie' ([Errno 11001] getaddrinfo failed)"))
https://subslikescript.commovie/A_Aa-5684466
-------- Link not working ----------

OR

In [23]:
import requests
from bs4 import BeautifulSoup

# Fetch the main page
root = 'https://subslikescript.com'
website = f'{root}/movies_letter-A'
result = requests.get(website, verify=False)

# Parse the main page
content = result.text
soup = BeautifulSoup(content, "lxml")

# Pagination
pagination = soup.find('ul', class_='pagination')
pages = pagination.find_all('li', class_='page-item')
last_page = pages[-2].text

links = []
for page in range(1, int(last_page) + 1)[:2]:  # [1, 138+1] to cover all pages
    result = requests.get(f'{website}?page={page}', verify=False)
    content = result.text
    soup = BeautifulSoup(content, "lxml")

    box = soup.find('article', class_="main-article")

    # Extract links from the main page
    for a in box.find_all('a', href=True):
        links.append(root + a['href'])

# Iterate over each link
for link in links:
    try:
        print(link)
        result = requests.get(link, verify=False)
        content = result.text
        soup = BeautifulSoup(content, "lxml")

        # Parse the content of the page
        box = soup.find('article', class_="main-article")
        title = box.find('h1').get_text()
        transcript = box.find('div', class_='full-script').get_text(strip=True, separator=' ')

        with open(f'{title}.txt', 'w', encoding='utf-8') as file:
            file.write(transcript)

    except Exception as e:
        print('-------- Link not working ----------')
        print(link)
        print(e)


C:\Users\DR E.A MAJI\anaconda33\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'subslikescript.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\DR E.A MAJI\anaconda33\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'subslikescript.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\DR E.A MAJI\anaconda33\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'subslikescript.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://subslikescript.commovie/A_1000000000000000_Ransom-4926482
-------- Link not working ----------
https://subslikescript.commovie/A_1000000000000000_Ransom-4926482
HTTPSConnectionPool(host='subslikescript.commovie', port=443): Max retries exceeded with url: /A_1000000000000000_Ransom-4926482 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001C5BED71C90>: Failed to resolve 'subslikescript.commovie' ([Errno 11001] getaddrinfo failed)"))
https://subslikescript.commovie/A_2nd_Chance-2150139
-------- Link not working ----------
https://subslikescript.commovie/A_2nd_Chance-2150139
HTTPSConnectionPool(host='subslikescript.commovie', port=443): Max retries exceeded with url: /A_2nd_Chance-2150139 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001C5BED71250>: Failed to resolve 'subslikescript.commovie' ([Errno 11001] getaddrinfo failed)"))
https://subslikescript.commovie/A_Aa-5684466
-------- Link not working ----------